# Prophet - Predecir el valor de Shiba Inu EUR


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Shiba Inu, una criptomoneda. 

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de Shiba Inu lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-09'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('SHIB-EUR',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
784,2022-11-04,$0.00,$0.00,$0.00,$0.00,$0.00,711013079
785,2022-11-05,$0.00,$0.00,$0.00,$0.00,$0.00,573140722
786,2022-11-06,$0.00,$0.00,$0.00,$0.00,$0.00,419840736
787,2022-11-07,$0.00,$0.00,$0.00,$0.00,$0.00,333636088
788,2022-11-08,$0.00,$0.00,$0.00,$0.00,$0.00,752498835


In [ ]:
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Validamos que no hay datos vacíos


In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del Shiba en ese momento
df = btc_df[['Date','Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

df = df.rename(columns={'Date':'ds','Open':'y'})
df.columns
# TAREA: Utiliza el diccionario para rembrar las columnas de df
df


,ds,y
0,2020-08-01,$0.00
1,2020-08-02,$0.00
2,2020-08-03,$0.00
3,2020-08-04,$0.00
4,2020-08-05,$0.00
...,...,...
784,2022-11-04,$0.00
785,2022-11-05,$0.00
786,2022-11-06,$0.00
787,2022-11-07,$0.00


In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
784,2022-11-04,$0.00
785,2022-11-05,$0.00
786,2022-11-06,$0.00
787,2022-11-07,$0.00
788,2022-11-08,$0.00


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Serie Temporal del Bitcoin"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"

m = Prophet(seasonality_mode= 'multiplicative')

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
#generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
#puedes usar la función del prophet make_future_dataframe()
#future = 
#future.tail()

SyntaxError: ignored

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
1149,2023-11-04
1150,2023-11-05
1151,2023-11-06
1152,2023-11-07
1153,2023-11-08


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2020-08-01
1,2020-08-02
2,2020-08-03
3,2020-08-04
4,2020-08-05
...,...
1149,2023-11-04
1150,2023-11-05
1151,2023-11-06
1152,2023-11-07


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1149,2023-11-04,$-0.00,$-0.00,$-0.00
1150,2023-11-05,$-0.00,$-0.00,$-0.00
1151,2023-11-06,$-0.00,$-0.00,$-0.00
1152,2023-11-07,$-0.00,$-0.00,$-0.00
1153,2023-11-08,$-0.00,$-0.00,$-0.00


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)